# 第五章：文件与IO

所有程序都要处理输入和输出。
这一章将涵盖处理不同类型的文件，包括文本和二进制文件，文件编码和其他相关的内容。对文件名和目录的操作也会涉及到。

## 5.1 读写文本数据

* 问题

你需要读写各种不同编码的文本数据，比如ASCII，UTF-8或UTF-16编码等。

* 解决方案

使用带有 `rt` 模式的 `open()` 函数读取文本文件。

In [1]:
# Read the entire file as s single string
with open('./passwd.txt', 'rt') as f:
    data = f.read()

# Iterate over the lines of the file
with open('./passwd.txt', 'rt') as f:
    for line in f:
        # process line
        pass

类似的，为了写入一个文本文件，使用带有 `wt` 模式的 `open()` 函数， 如果之前文件内容存在则清除并覆盖掉。如下所示：
```python
# Write chunks of text data
with open('somefile.txt', 'wt') as f:
    f.write(text1)
    f.write(text2)

# Redirected print statement
with open('somefile.txt', 'wt') as f:
    print(line1, file=f)
    print(line2, file=f)
```

如果是在已存在文件中添加内容，使用模式为 `at` 的 `open()` 函数。

文件的读写操作默认使用系统编码，可以通过调用 `sys.getdefaultencoding()` 来得到。

在大多数机器上面都是`utf-8`编码。如果你已经知道你要读写的文本是其他编码方式，那么可以通过传递一个可选的 `encoding` 参数给 `open()` 函数。
```python
with open('somefile.txt', 'rt', encoding='latin-1') as f:
```

Python支持非常多的文本编码。几个常见的编码是ascii, latin-1, utf-8和utf-16。

在web应用程序中通常都使用的是UTF-8。

ascii对应从U+0000到U+007F范围内的7位字符。
latin-1是字节0-255到U+0000至U+00FF范围内Unicode字符的直接映射。

当读取一个未知编码的文本时使用latin-1编码永远不会产生解码错误。

使用latin-1编码读取一个文件的时候也许不能产生完全正确的文本解码数据，但是它也能从中提取出足够多的有用数据。同时，如果你之后将数据回写回去，原先的数据还是会保留的。

* 讨论

读写文本文件一般来讲是比较简单的。但是也几点是需要注意的。
* 首先，在例子程序中的`with`语句给被使用到的文件创建了一个上下文环境，`with` 控制块结束时，文件会自动关闭。

你也可以不使用 `with` 语句，但是这时候你就必须记得手动关闭文件：
```python
# 手动关闭文件
f = open('somefile.txt', 'rt')
data = f.read()
f.close()
```

另外一个问题是关于换行符的识别问题，在Unix和Windows中是不一样的(分别是 \n 和 \r\n )。

默认情况下，Python会以统一模式处理换行符。这种模式下，在读取文本的时候，Python可以识别所有的普通换行符并将其转换为单个 \n 字符。

类似的，在输出时会将换行符 \n 转换为系统默认的换行符。 如果你不希望这种默认的处理方式，可以给 open() 函数传入参数 newline='' ，就像下面这样：

```python
# Read with disabled newline translation
with open('somefile.txt', 'rt', newline='') as f:
    pass
```

为了说明两者之间的差异，下面我在Unix机器上面读取一个Windows上面的文本文件，里面的内容是 `hello world!\r\n`：
```python
# Newline translation enabled (the default)
f = open('hello.txt', 'rt')
f.read()
# 'hello world!\n'

# Newline translation disabled
g = open('hello.txt', 'rt', newline='')
g.read()
# 'hello world!\r\n'
```

最后一个问题就是文本文件中可能出现的编码错误。 但你读取或者写入一个文本文件时，你可能会遇到一个编码或者解码错误。比如：
```python
>>> f = open('sample.txt', 'rt', encoding='ascii')
>>> f.read()
Traceback (most recent call last):
    File "<stdin>", line 1, in <module>
    File "/usr/local/lib/python3.3/encodings/ascii.py", line 26, in decode
        return codecs.ascii_decode(input, self.errors)[0]
UnicodeDecodeError: 'ascii' codec can't decode byte 0xc3 in position
12: ordinal not in range(128)
>>>
```

如果出现这个错误，通常表示你读取文本时指定的编码不正确。

最好仔细阅读说明并确认你的文件编码是正确的(比如使用UTF-8而不是Latin-1编码或其他)。

如果编码错误还是存在的话，你可以给 `open()` 函数传递一个可选的 `errors` 参数来处理这些错误。 下面是一些处理常见错误的方法：
```python
>>> # Replace bad chars with Unicode U+fffd replacement char
>>> f = open('sample.txt', 'rt', encoding='ascii', errors='replace')
>>> f.read()
'Spicy Jalape?o!'
>>> # Ignore bad chars entirely
>>> g = open('sample.txt', 'rt', encoding='ascii', errors='ignore')
>>> g.read()
'Spicy Jalapeo!'
>>>
```

如果你经常使用 `errors` 参数来处理编码错误，可能会让你的代码变得很混乱。

对于文本处理的首要原则是**确保你总是使用的是正确编码**。当模棱两可的时候，就使用默认的设置(通常都是UTF-8)。

## 5.2 打印输出至文件中

* 问题

你想将 `print()` 函数的输出重定向到一个文件中去。

* 解决方案

在 `print()` 函数中指定 `file` 关键字参数，像下面这样：
```python
with open('d:/work/test.txt', 'wt') as f:
    print('Hello World!', file=f)
```

* 讨论
文件必须是以文本模式打开。如果文件是二进制模式的话，打印就会出错。

In [2]:
with open('./passwd.txt', 'wt') as f:
    print("Hello World!", file=f)  # 将"Hello World!" 保存在passed.txt中

## 5.3 使用其他分隔符或行终止符打印

* 问题

你想使用 `print()` 函数输出数据，但是想改变默认的分隔符或者行尾符。

* 解决方案

可以使用在 `print()` 函数中使用 `sep` 和 `end` 关键字参数，以你想要的方式输出。


In [3]:
print("ACME", 50, 91.5)

ACME 50 91.5


In [4]:
print("ACME", 50, 91.5, sep=',')

ACME,50,91.5


In [5]:
print("ACME", 50, 91.5, sep=',', end='!!\n')

ACME,50,91.5!!


使用 `end`` 参数也可以在输出中禁止换行。

In [6]:
for i in range(5):
    print(i)

0
1
2
3
4


In [8]:
for i in range(5):
    print(i, end='')

01234

* 讨论

当你想使用**非空格分隔符**来输出数据的时候，给 `print()` 函数传递一个 `sep` 参数是最简单的方案。

有时候你会看到一些程序员会使用 `str.join()` 来完成同样的事情。

In [9]:
print(','.join(('ACME','50','91.5')))

ACME,50,91.5


`str.join()` 的问题在于它仅仅适用于字符串。这意味着你通常需要执行另外一些转换才能让它正常工作。比如：
```python
# 50：int; 91.5: float is not str. must convert it to str type
>>> row = ('ACME', 50, 91.5)
>>> print(','.join(row))
Traceback (most recent call last):
    File "<stdin>", line 1, in <module>
TypeError: sequence item 1: expected str instance, int found
>>> print(','.join(str(x) for x in row))
ACME,50,91.5
>>>
```

In [10]:
row = ('ACME', 50, 91.5)
print(','.join(row))

TypeError: sequence item 1: expected str instance, int found

In [11]:
print(','.join(str(x) for x in row))

ACME,50,91.5


In [12]:
# 你当然可以不用那么麻烦，只需要像下面这样写：
print(*row, sep=',')

ACME,50,91.5


## 5.4 读写字节数据

* 问题
你想读写二进制文件，比如图片，声音文件等等。

* 解决方案

使用模式为 `rb` 或 `wb` 的 `open()` 函数来读取或写入二进制数据。比如：
```python
# Read the entire file as a single byte string
with open('somefile.bin', 'rb') as f:
    data = f.read()

# Write binary data to a file
with open('somefile.bin', 'wb') as f:
    f.write(b'Hello World')
```

在读取二进制数据时，需要指明的是所有返回的数据都是字节字符串格式的，而不是文本字符串。

类似的，在写入的时候，必须保证参数是以字节形式对外暴露数据的对象(比如字节字符串，字节数组对象等)。

* 讨论

在读取二进制数据的时候，字节字符串和文本字符串的语义差异可能会导致一个潜在的陷阱。

特别需要注意的是，索引和迭代返回的是字节的值而不是字节字符串。

In [13]:
# Text string
t = 'Hello World'
t[0]

'H'

In [14]:
for c in t:
    print(c)

H
e
l
l
o
 
W
o
r
l
d


In [15]:
# Byte string
b = b'Hello World'
b[0]

72

In [16]:
for c in b:
    print(c)

72
101
108
108
111
32
87
111
114
108
100


如果你想从二进制模式的文件中读取或写入文本数据，必须确保要进行解码和编码操作。比如：

```python
with open('somefile.bin', 'rb') as f:
    data = f.read(16)
    text = data.decode('utf-8')

with open('somefile.bin', 'wb') as f:
    text = 'Hello World'
    f.write(text.encode('utf-8'))
```

二进制I/O还有一个鲜为人知的特性就是数组和C结构体类型能直接被写入，而不需要中间转换为自己对象。比如：

In [18]:
import array

nums = array.array('i', [1, 2, 3, 4])

with open('data.bin', 'wb') as f:
    f.write(nums)

这个适用于任何实现了被称之为”缓冲接口”的对象，这种对象会直接暴露其底层的内存缓冲区给能处理它的操作。 二进制数据的写入就是这类操作之一。

很多对象还允许通过使用文件对象的 `readinto()` 方法直接读取二进制数据到其底层的内存中去。比如：

In [19]:
import array
a = array.array('i', [0, 0, 0, 0, 0, 0, 0, 0])
with open('data.bin', 'rb') as f:
    f.readinto(a)

In [20]:
a

array('i', [1, 2, 3, 4, 0, 0, 0, 0])

但是使用这种技术的时候需要格外小心，因为它通常**具有平台相关性**，并且可能会**依赖字长和字节顺序（高位优先和低位优先）。**

可以查看5.9小节中另外一个读取二进制数据到可修改缓冲区的例子。